In [ ]:
#coding:utf-8
import warnings
warnings.filterwarnings("ignore")
import numpy as np    #numpy计算包
import codecs   #codecs提供的open方法来指定打开的文件的语言编码，它会在读取的时候自动转换为内部unicode 
import pandas as pd  
import matplotlib.pyplot as plt #matplotlib作为常用的可视化工具
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)
from wordcloud import WordCloud #词云包
import re
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
nltk.download('popular')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from google.colab import drive

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/CS4248/fulltrain.csv'
colnames=['Label', 'Text'] 
df = pd.read_csv(file_path, names=colnames, header=None)
df.head()

,Label,Text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [ ]:
df = df.dropna()
content=df["Text"].values.tolist() # 使用pandas读入文件，并转为list保存
segment=[]
# line = content[0]
# print(line[4:])
stop_words = set(stopwords.words('english'))
for line in content:
    line.lower()
    try: # 对每条数据进行分词
        word_tokens = word_tokenize(line[4:])
        segs = [w for w in word_tokens if not w.lower() in stop_words]
        # doc = nlp(line[3:])
        # # Get the individual text tokens
        # segs = [ t.text for t in doc]
        for seg in segs:
            if len(seg)>2 and seg!='\r\n': # 除去长度为1的词、换行符
                segment.append(seg)
    except:
        print(line)
        continue

In [ ]:
print(segment[:5])

['ttle', 'less', 'decade', 'ago', 'hockey']


In [ ]:
words_df=pd.DataFrame({'segment':segment})
#words_df.head()
# stopwords=pd.read_csv("stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')#quoting=3全不引用
# #stopwords.head()
# words_df=words_df[~words_df.segment.isin(stopwords.stopword)] # 把包含在停用词表里的词语去掉 
negation_words = ['no', 'not', 'failed', 'unable', 'only', 'refused']
words_df=words_df[words_df.segment.isin(negation_words)] # 把包含在停用词表里的词语去掉 
print(words_df.count())

segment    6862
dtype: int64


In [ ]:
words_df.head()

,segment
591,unable
873,failed
1286,unable
1299,failed
4332,unable


In [ ]:
words_stat=words_df.groupby(by=['segment'])['segment'].agg(count=np.size) # 统计词频数，并添加计数列
words_stat=words_stat.reset_index().sort_values(by=["count"],ascending=False) # 对词频进行排序
words_stat.head(10)

,segment,count
0,failed,3309
1,refused,1841
2,unable,1712


In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
from imageio import imread
matplotlib.rcParams['figure.figsize'] = (12.0, 12.0)
from wordcloud import WordCloud,ImageColorGenerator
# bimg=imread('img/entertainment.jpeg')
wordcloud=WordCloud(background_color="white", stopwords =STOPWORDS, font_path='/content/simhei.ttf',max_font_size=200) # 设定自定义背景图片
word_frequence = {x[0]:x[1] for x in words_stat.head(1000).values}
wordcloud=wordcloud.fit_words(word_frequence)
# bimgColors=ImageColorGenerator(bimg)
# plt.axis("off")
plt.imshow(wordcloud)

OSError: ignored

In [ ]:
import torch
import torchtext
from torchtext import data
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data.utils import ngrams_iterator

import spacy # I'll be making use of spacy for text preprocessing
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string, re

from torch.autograd import Variable
import time
import copy
from torch.optim import lr_scheduler
from torch.utils.data.dataset import random_split
from sklearn.model_selection import train_test_split
from torchtext.vocab import Vectors, GloVe
from matplotlib.pyplot import plot, hist, xlabel, legend
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import logging
import warnings
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt='%H:%M:%S', level=logging.INFO)
warnings.filterwarnings("ignore")

In [ ]:
train_df = pd.read_csv("/content/fulltrain.csv", encoding='utf-8')
train_df.head()

In [ ]:
from collections import defaultdict
train1_df = train[train["Verdict"]==1]
train2_df = train[train["Verdict"]==2]
train3_df = train[train["Verdict"]==3]
train4_df = train[train["Verdict"]==4]


## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

## custom function for horizontal bar chart ##
def horizontal_bar_chart(df, color):
    trace = go.Bar(
        y=df["word"].values[::-1],
        x=df["wordcount"].values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

## Get the bar chart from sincere questions ##
freq_dict = defaultdict(int)
for sent in train0_df["Text"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace0 = horizontal_bar_chart(fd_sorted.head(50), '#bccbde')

## Get the bar chart from insincere questions ##
freq_dict = defaultdict(int)
for sent in train1_df["Text"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace1 = horizontal_bar_chart(fd_sorted.head(50), '#c2dde6')

## Get the bar chart from insincere questions ##
freq_dict = defaultdict(int)
for sent in train_1_df["Text"]:
    for word in generate_ngrams(sent):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
trace_1 = horizontal_bar_chart(fd_sorted.head(50), '#c2dde6')


# Creating two subplots
fig = tools.make_subplots(rows=1, cols=3, vertical_spacing=0.04,
                          subplot_titles=["Frequent words of important facts", 
                                          "Frequent words of unimportant statements",
                                          "Frequent words of non-factual statements"])
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace_1, 1, 3)
fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Word Count Plots")
py.iplot(fig, filename='word-plots')
